In [2]:
import gym
import ns3gym
from stable_baselines3 import PPO
from ns3gym import ns3env

In [ ]:
env=gym.make("ns3-v0")


Got new port for ns3gm interface:  9517
ns3 file not found. Quitting...


SystemExit: 